In [8]:
# 전이학습
# 사전학습된 네트워크의 가중치를 사용하는 것입니다.
# 1) 모델을 변형하지 않고 사용하는 방법
# 2) 모델의 분류기를 재학습하는 방법
# 3) 모델의 일부를 동결해제해서 재학습하는 방법

# 전이학습은 주로 ImageNet 데이터를 학습시킨 가중치를 사용하거나 본인만의 데이터셋을 사용해서
# 재학습을 진행하는데, 이것을 미세조정이라고 합니다.
# https://keras.io/applications

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16


# weights = imagenet 데이터를 학습시킨 가중치의 사용 여부를 결정,
# 기본값은 None, 가중치를 사용하고 싶다면 'imagenet' 전달해야함

# input_shape : 입력 데이터의 형태를 전달

# include_top : 모델의 분류기층을 포함해서 구성할지 전달합니다.
vgg16 = VGG16(weights='imagenet',
             input_shape=(32, 32,3), include_top = False)

model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 14,849,610
Trainable params: 14,849,098
Non-trainable params: 512
________________________________________

In [9]:
from tensorflow.keras.applications import *

mobilenet = MobileNet(weights = None, input_shape=(32,32,3),
                     include_top = True)

In [11]:
# 모델 동결 해제
for layer in vgg16.layers[:-4]:
    layer.trainable= False
    
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

AttributeError: module 'tensorflow.keras.applications.vgg16' has no attribute 'layers'

In [ ]:
# 데이터 준비 -> 전이 학습 사용 -> 사전 학습 모델 사용
# -> 동결 해제 -> 모델 구성 및 학습

In [1]:
from tensorflow.keras.datasets import cifar10

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

import matplotlib.pyplot as plt
import numpy as np

x_mean = np.mean(x_train, axis=(0,1,2))
x_std = np.std(x_train, axis=(0,1,2))

x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

from sklearn.model_selection import train_test_split

x_train, x_val, t_train, t_val = train_test_split(x_train, t_train,
                                                 test_size=0.3)

In [5]:

from tensorflow.keras.applications import VGG16
vgg16 = VGG16(weights='imagenet',
             input_shape=(32, 32,3), include_top = False)

vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip = True,
                                  zoom_range = 0.2,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  rotation_range=30,
                                  fill_mode = 'nearest')

val_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow(x_train,t_train,
                                    batch_size = batch_size)

val_generator = val_datagen.flow(x_val, t_val,
                                batch_size = batch_size)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

# 모델 동결 해제
for layer in vgg16.layers[:-4]:
    layer.trainable= False
    
    
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(1e-4),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

def get_step(train_len, batch_size):
    if(train_len % batch_size > 0):
        return train_len // batch_size + 1
    else:
        return train_len // batch_size
    
history = model.fit(train_generator,
                   epochs = 100,
                   steps_per_epoch = get_step(len(x_train), batch_size),
                   validation_data=val_generator,
                   validation_steps=get_step(len(x_val), batch_size))

Epoch 1/100
1094/1094 [==============================] - 387s 354ms/step - loss: 1.1268 - acc: 0.1005 - val_loss: 0.9990 - val_acc: 0.0925
Epoch 2/100
1094/1094 [==============================] - 386s 353ms/step - loss: 0.9196 - acc: 0.0991 - val_loss: 0.9361 - val_acc: 0.0608
Epoch 3/100
1094/1094 [==============================] - 377s 344ms/step - loss: 0.8502 - acc: 0.0996 - val_loss: 0.7371 - val_acc: 0.0907
Epoch 4/100
1094/1094 [==============================] - 386s 352ms/step - loss: 0.7969 - acc: 0.1009 - val_loss: 0.7770 - val_acc: 0.0945
Epoch 5/100
1094/1094 [==============================] - 386s 353ms/step - loss: 0.7628 - acc: 0.1003 - val_loss: 0.7914 - val_acc: 0.1107
Epoch 6/100
 844/1094 [======================>.......] - ETA: 1:22 - loss: 0.7398 - acc: 0.1007

In [ ]:
CIFAR100 훈련 -> 에폭 10
Dense층을 통한 모델의 뷴류기 직접 구성
사전학습 ResNet, Xception
전이 학습a
